# Script confronto Modelli Classici

# Import librerie

In [0]:
import json
import joblib
import time
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, cross_val_score,StratifiedKFold
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import PCA

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import Dataset

In [0]:
my_dump = joblib.load('/content/drive/My Drive/Text Mining/df_lemma2.joblib')

In [68]:
my_dump.tail()

,Unnamed: 0,overall,reviewText,sentiment
199995,71359,1.0,perfect everything need add water pond,1
199996,71360,1.0,use rain water change,0
199997,71361,1.0,good stuff,1
199998,71362,1.0,much well economical maintenance tetra easy ...,3
199999,71363,1.0,honestly far well product use pond four hund...,1


In [0]:
X_train, X_test, y_train, y_test = train_test_split(my_dump['reviewText'], 
                                                    my_dump['overall'], 
                                                    test_size=0.2, 
                                                    random_state=8)

# Tf-*idf*

In [0]:
ngram_range = (1,2)
max_features = 1000

In [0]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)

In [0]:
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
#print(features_train.shape)

# Embedding

## PCA

In [0]:
pca = PCA(0.95,svd_solver ='full',random_state=42)

In [0]:
pca = pca.fit(features_train,labels_train)
X_train= pca.transform(features_train)

In [38]:
X_train.shape

(40000, 823)

In [62]:
features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(40000, 1000)


In [0]:
X_test=pca.transform(features_test)

## LDA

In [0]:
lda=LatentDirichletAllocation(n_components=256,n_jobs=-1)

In [0]:
start = time.time()
LDA = lda.fit(features_train,y_train)
end = time.time() 

In [0]:
end-start

In [0]:
X_train_lda=LDA.transform(features_train)

In [0]:
X_test_lda=LDA.transform(features_test)

# Modelli

In [0]:
Results = {}

## SVC

In [0]:
n_estimators = 10
clf = BaggingClassifier(SVC(kernel='linear',probability=True), max_samples=1.0 / n_estimators, n_estimators=n_estimators,n_jobs=-1)

In [0]:
start = time.time()
clf.fit(X_train, y_train)
end = time.time()

In [54]:
print((end-start)/60)

77.21823565165202


In [55]:
start = time.time()
score = clf.score(X_test,y_test)
end = time.time()
print((end-start)/60)

22.21394064426422


In [0]:
Results['SVC']=score

In [0]:
with open('/content/drive/My Drive/Text Mining/Model_Selection_res.json', 'w') as json_file:
  json.dump(Results, json_file)

In [0]:
#model_evaluator(clf)

[0.78496438 0.77817364 0.78548659]
Accuracy of Model with Cross Validation is: 0.78% (+/- 0.00%)


## Random Forest

In [0]:
clf = RandomForestClassifier(n_estimators=80,n_jobs=-1)
clf.fit(X_train, y_train)
#clf.fit(X_train_lda, y_train)

In [0]:
score = clf.score(X_test,y_test)
#score = clf.score(X_test_lda,y_test)

In [0]:
Results['RF'] = score

In [0]:
with open('/content/drive/My Drive/Text Mining/Model_Selection_res.json', 'w') as json_file:
  json.dump(Results, json_file)

## KNN

In [0]:

with open('/content/drive/My Drive/Text Mining/Model_Selection_res.json') as json_file:
    Results = json.load(json_file)

In [42]:
clf = KNeighborsClassifier(n_neighbors=5,n_jobs=-1)
clf.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
                     weights='uniform')

In [0]:
score = clf.score(X_test,y_test)
Results['KNN'] = score

In [0]:
with open('/content/drive/My Drive/Text Mining/Model_Selection_res.json', 'w') as json_file:
  json.dump(Results, json_file)


## Multinomial Naive Bayes

The multinomial Naive Bayes classifier is suitable for classification with discrete features (e.g., word counts for text classification). The multinomial distribution normally requires integer feature counts. However, in practice, fractional counts such as tf-idf may also work.


Don't use Naive Bayes with SVD or other matrix factorization (es PCA) because Naive Bayes based on applying Bayes' theorem with strong (naive) independence assumptions between the features.


In [61]:
clf = MultinomialNB()
clf.fit(features_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
score=clf.score(features_test,y_test)

In [0]:
Results['Multinomial_Naive_Bayes'] = score

In [65]:
Results

{'KNN': 0.6248,
 'Multinomial_Naive_Bayes': 0.8102,
 'RF': 0.783875,
 'SVC': 0.8222083333333333}